In [3]:
pip install pandas pyodbc


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import pyodbc

# 1. قراءة ملف CSV
df = pd.read_csv(r"C:\Book_multisources_clean.csv", encoding="utf-8")

# 2. ربط مع SQL Server Express
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=HABIBAELNAMAKY\\SQLEXPRESS;"   # ✅ هنا استخدمنا \\
    "Database=BOOK;"                       # اسم قاعدة البيانات
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

# 3. إنشاء الجدول (لو موجود بيتحذف ويتعمل تاني)
cursor.execute("""
IF OBJECT_ID('Books_Final', 'U') IS NOT NULL
    DROP TABLE Books_Final;
CREATE TABLE Books_Final (
    BookID INT IDENTITY(1,1) PRIMARY KEY,
    BookTitle NVARCHAR(500),
    Price FLOAT NULL,
    Availability INT NULL,
    Rating INT NULL,
    ImageURL NVARCHAR(1000),
    BookLink NVARCHAR(1000),
    Category NVARCHAR(255),
    BookTitle_Cleaned NVARCHAR(500),
    Price_Normalized FLOAT NULL,
    Category_Encoded FLOAT NULL
)
""")
conn.commit()

# 4. إدخال البيانات صف بصف
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Books_Final (
            BookTitle, Price, Availability, Rating, ImageURL, BookLink,
            Category, BookTitle_Cleaned, Price_Normalized, Category_Encoded
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, 
    row["Book Title"], 
    float(row["Price"]) if pd.notna(row["Price"]) else None,
    int(row["Availability"]) if pd.notna(row["Availability"]) else None,
    int(row["Rating"]) if pd.notna(row["Rating"]) else None,
    row["Image URL"], 
    row["Book Link"], 
    row["Category"],
    row["BookTitle_Cleaned"],
    float(row["Price_Normalized"]) if pd.notna(row["Price_Normalized"]) else None,
    float(row["Category_Encoded"]) if pd.notna(row["Category_Encoded"]) else None
    )

conn.commit()
cursor.close()
conn.close()

print("✅ تم إدخال البيانات في Books_Final بنجاح!")


✅ تم إدخال البيانات في Books_Final بنجاح!
